# WS44 - High throughput & automated data analysis and data management workflow with Cellprofiler and OMERO

### Introduction

In this workshop we will use this Jupyter Notebook to load image data from OMERO, feed them into a Cellprofiler pipeline and automatically upload the resulting images and measurements. The uploaded data will also be annotated using tags and key:value pairs.

### Tasks during the workshop
1.     (Data import to OMERO and preparation for analysis.)
2.  	Automated data download/injection into analysis pipeline
3.  	Automated data analysis using image analysis pipelines (e.g., Cellprofiler)
4.  	Upload of the resulting images (including tags and metadata) and measurement results (omero.tables)
5.  	Explorative data analysis using omero.parade

### Aims of this workshop:

- learn to analyze provided example datasets
- execute the full workflow
- perform easy adjustments of the pipeline 
- generation of new projects/datasets
- key:value pair annotation
- file tagging
- explorative data analysis using omero.parade/omero.parade-crossfilter

### Dataset

The data used in this workshop is derived from Pascual-Vargas et al., Sci Data, 2017
"RNAi screens for Rho GTPase regulators of cell shape and YAP/TAZ localisation in triple negative breast cancer"
DOI: 10.1038/sdata.2017.18

The data is publicly available in the Image Data Resource (idr0028)
https://idr.openmicroscopy.org/webclient/?show=screen-1651

The datasets contain RNAi screens of cancer cells that were stained with Hoechst (Nuclei), Tubulin, Actin and Yap/Taz.


### Licenses & Code

The code presented here is partially based on the following scripts and resources:

- Omero Dataset_To_Plate.py script by Will Moore, OME Team, Copyright © 2006-2014 University of Dundee. All rights reserved. Source: https://github.com/ome/omero-scripts/blob/68c7505e62115e9c086a8e5a1d3edc1d4aff35f3/omero/util_scripts/Dataset_To_Plate.py
<br>

- InjectImage module for Cellprofiler. Copyright © 2020-2021 University of Dundee. All rights reserved. Source: https://omero-guides.readthedocs.io/en/latest/cellprofiler/docs/index.html; https://github.com/ome/omero-guide-cellprofiler
<br>

- General Omero-Python API documentation, Source: https://omero-guides.readthedocs.io/en/latest/python/docs/gettingstarted.html
<br>

- Cellprofiler Python API, Copyright © 2003 - 2021 Broad Institute, Inc. All rights reserved.Source: https://github.com/CellProfiler/CellProfiler/wiki/CellProfiler-as-a-Python-package
<br>

- ezomero (https://github.com/TheJacksonLaboratory/ezomero) 
<br>


### Imports

In [1]:
#Cellprofiler
import cellprofiler_core.pipeline
import cellprofiler_core.preferences
import cellprofiler_core.utilities.java
import cellprofiler.modules
import cellprofiler_core.image
import cellprofiler_core.measurement
import cellprofiler_core.object
import cellprofiler_core.workspace
from cellprofiler_core.modules.injectimage import InjectImage


#Omero
import ezomero
from myconfig import OMEROUSER, OMEROPASS, OMEROPORT, OMEROHOST
from omero.model import OriginalFileI, PlateI, ScreenPlateLinkI, ScreenI, ImageAnnotationLinkI, ImageI
from omero.rtypes import rint, rlong, rstring, robject, unwrap
from omero.grid import DoubleColumn, ImageColumn, LongColumn, WellColumn, StringColumn, FileColumn, RoiColumn
from omero.constants.namespaces import NSBULKANNOTATIONS
from omero.gateway import FileAnnotationWrapper


#Other
import h5py
import pandas as pd
import skimage.io
import os
import pathlib
import pickle
import tempfile
import skimage
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime
import warnings
import time
import glob
import PIL
import re
import cv2
import json
import shutil
import getpass
import numpy.ma as ma
from skimage.measure import find_contours

#Own functions
import CP_Omero_helper as cp_omero

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\WS44\lib\site-packages\csbdeep\models\__init__.py", line 5, in <module>
    import tensorflow
ModuleNotFoundError: No module named 'tensorflow'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\WS44\lib\site-packages\cellprofiler_core\utilities\core\modules\__init__.py", line 71, in add_module
    m = __import__(mod, globals(), locals(), ["__all__"], 0)
  File "C:\Program Files\Cellprofiler_updated\Cellprofiler\cellprofiler_plugins\runstardist.py", line 5, in <module>
    from stardist.models import StarDist2D, StarDist3D
  File "C:\ProgramData\Anaconda3\envs\WS44\lib\site-packages\stardist\models\__init__.py", line 3, in <module>
    from .model2d import Config2D, StarDist2D, StarDistData2D
  File "C:\ProgramData\Anaconda3\envs\WS44\lib\site-packages\stardist\models\model2d.py", line 8, in <module>
    from csbdeep.model

### Parameters

In the code block below, you will add specific analysis paramters, such as the screen and plate id, you would like to image, as well as filepaths and other settings.

In [ ]:
# Login to OMERO
#OMEROUSER = input(f"Enter username: \t")
#OMEROPASS = getpass.getpass(prompt = f"Enter password: \t")


#OMEROHOST = ''
#OMEROPORT = 
#OMEROWEB = ''

In [3]:
# Connection Check:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)
print(conn.isConnected())

True


In [4]:
# OMERO IDs
screen_id = 852 #Insert ID of dataset that you want to analyse
plate_id =  765#Insert corresponding plate ID
project_id = 4118 #Project ID for temp - dataset
selected_well = "E5" # Insert well you want to analyse
tag_owner_id = 2607  # To keep the omero server clean, we will all use tags from 1 tag owner. Otherwise everyone would produce their own tags.

# Pipeline
pipe_dir = r"C:\Users\MiN_Acc1\Documents\GitHub\TiM23_WS44_HTPImageAnalysis_Omero\Cellprofiler\Cellprofiler Pipeline\IRD0028_CP_v8_noGPU.cppipe" #Insert directory of pipeline including name of pipeline

# Input and saving directories:
output_dir = "temp_dir" 
# if you want to use a temporary directory that is automatically created use: "output_dir = 'temp_dir'"

# Cellprofiler-settings
# (maybe remove)
overwrite_results = 'Yes'  # If yes, data present in the output folder will be overwritten
output_file_format = None  # 'npy' for numpy array, 'tiff' for image (label images: 16-bit floating point), put None if you want to keep the fileformats specified in your pipeline
plugin_directory = ""

# Name of the new dataset to which the label images will be uploaded
new_plate_name = "Results_"
append_original_plate_name = True # False

# Specify the channels that should be used for segmentation and analysis
# Same names as in CP pipeline!
ch1 = "Nuclei" #Nuclei segmentation
ch2 = "Actin" #Actin (cell body) segmentation
ch3 = "Tubulin"
ch4 = "YapTaz" #YapTaz for analysis
# ... expand if you have more channel .. ch5 = xx

channels = [ch1, ch2, ch3, ch4]

In [5]:
# Key:Value Pairs - Add the annotations you would like to pass with your analysis results

# Coud be excel sheet or textfile, or written here.
annotation_dict= {
"Goal & Description" : "We segment the cells and calculate the YAP/Taz ratio in the nuclei.", 
"Software" : "Cellprofiler",
"Software Version": "4.2.5",
"Segmentation Algorithm": "Cellpose",
"Segmentation Algorithm Version": "?",
"Models": "nuclei, cyto2",
"eLabTFW": "https://eln.uni-muenster.de/experiments.php?mode=edit&id=71",
}

## 1. Perform Cellprofiler Analysis

In this part we will obtain the image data from omero, inject it into the cellprofiler analysis pipeline and perform the image analysis. Results will be saved on disk in the specified output folder.

In [6]:
### Prepare Cellprofiler

#Set output directory
if output_dir == "temp_dir":
    temp_dir = tempfile.mkdtemp()  # Creates a temporary directory
    temp_path = os.path.normcase(temp_dir)
    saving_path = pathlib.Path(temp_path).absolute()
else:
    saving_path = pathlib.Path(output_dir).absolute()

cellprofiler_core.preferences.set_default_output_directory(f"{saving_path}")
print(f"Data will be saved to: {saving_path}")    


# Set-Up Cellprofiler
cellprofiler_core.preferences.set_headless() # The headless mode runs cellprofiler without use of the GUI. 
cellprofiler_core.preferences.set_plugin_directory(plugin_directory) # Sets the plugin directory that contains the cellpose module plugin
cellprofiler_core.preferences.set_max_workers(1) # You can increase the number of workers depending on your computer/server hardware.


#Start the Java VM
cellprofiler_core.utilities.java.start_java()

Data will be saved to: c:\users\min_acc1\appdata\local\temp\5\tmpn16qqres


In [7]:
# Here we load the pipeline and adjust it to work with Omero. 

pipeline = cp_omero.load_pipeline(pipe_dir)
pipeline = cp_omero.adjust_pipeline(pipeline, overwrite_results, output_file_format) 

Remove module:  Images
Remove module:  Metadata
Remove module:  NamesAndTypes
Remove module:  Groups
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in output file format.
No changes in o

In [8]:
# Start Analysis

# We define a timer to track how long the analysis will take.
start_time = datetime.now().strftime("%H:%M:%S")


# We connect to omero and get the plate we want to analyse
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True) # Connection to Omero
plate = conn.getObject("Plate", plate_id) # Gets the plate you want to analyse


# Start of the analysis
print(f"You are analyzing the well {selected_well}.")
measurements = {}
df_features=pd.DataFrame()


# The code will loop through each well and perform the analysis. 
wells = list(plate.listChildren())

for count, well in enumerate(wells):
    if (well.row, well.column) == (cp_omero.well_name_to_position(selected_well)):  #For workshop purposes we only select one well to analse
        print(well.row, well.column)

    # Load a single Image per Well
        index = well.countWellSample() # Will analyse all images in the well
        index = 1 # Will analyse 1 image in the well (omit if you want to analyse all images)

        for i in range(0, index):
            image = well.getImage(i)
            image_id = image.getId()
            pixels = image.getPrimaryPixels()
            size_c = image.getSizeC()

            # For each Image in OMERO, we copy pipeline, add the image_id to the Saving and Export Modules. 
            pipeline_copy = pipeline.copy()

            # Find the SaveImages modules and update its settings          
            pipeline_copy = cp_omero.update_save_images_module_setting(pipeline, image_id)       
                            
            # Find the ExportToSpreadsheet module and update its settings
            pipeline_copy = cp_omero.update_export_module_setting(pipeline, image_id)

            # Inject image for each channel into the pipeline.
            for c in range(0, size_c):
                plane = pixels.getPlane(0, c, 0)
                image_name = image.getName()
                image_id = image.getId()

                # Name of the channel expected in the pipeline
                if c == 0:
                    image_name = ch1
                if c == 1:
                    image_name = ch2
                if c == 2:
                    image_name = ch3
                if c == 3:
                    image_name = ch4
                inject_image_module = InjectImage(image_name, plane)
                inject_image_module.set_module_num(1)
                pipeline_copy.add_module(inject_image_module)

            # Here we run the pipeline on our image.
            output_measurements = pipeline_copy.run()

            # Here we process the measurement results
            measurements[image_id] = output_measurements
            feature_meas = output_measurements.compute_aggregate_measurements(1, aggs=None)
            df_feature = pd.DataFrame(feature_meas, index=[image_id])
            df_features = pd.concat([df_features,df_feature])
            print(f"ImageID: {image_id} :  finished")

df_features["Image_ID"] = df_features.index
df_features.to_csv(os.path.join(saving_path,"features_summary.csv")) #Saving the results

# Timer
end_time = datetime.now().strftime("%H:%M:%S")

print(f"Pipeline finished: {len(measurements)} images analysed")

print(f"Analysis started: {start_time}")
print(f"Analysis finished: {end_time}")

You are analyzing the well E5.
4 4


dtype.py (482): Downcasting int32 to uint16 without scaling because max value 13 fits in uint16
saveimages.py (791): c:\users\min_acc1\appdata\local\temp\5\tmpn16qqres\186174_NucLabel.tiff is a low contrast image
dtype.py (482): Downcasting int32 to uint16 without scaling because max value 15 fits in uint16
saveimages.py (791): c:\users\min_acc1\appdata\local\temp\5\tmpn16qqres\186174_CytoLabel.tiff is a low contrast image


ImageID: 186174 :  finished
Pipeline finished: 1 images analysed
Analysis started: 15:14:20
Analysis finished: 15:15:03


In [ ]:
# parallelization: run pipeline with run_image_set ?

In [9]:
# Saving updated Pipeline
with open(os.path.join(str(saving_path), 'Omero_Pipeline.cppipe'), "wt") as fd:
    #if fd_or_filename.endswith(".json"):
    pipeline_copy.dump(fd, save_image_plane_details=False)

In [ ]:
#conn.close()

## 2. Upload Results To Omero

We will now upload the results to omero.  

We will first create a new screen and plate to host the resulting images. <br>
Then we will derive image information (parent ID and appendix) from the file name. <br>
The images will be updated to a (temporary) dataset. <br>
Finally, all images will be distributed on the new results plate in the corresponding wells. <br>

In [10]:
############# 1. Creation of plate that hosts the results #############
#conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)
screen = conn.getObject("Screen", screen_id)


# Create new plate
plate = conn.getObject("Plate", plate_id)
if append_original_plate_name:
    plate_name = new_plate_name + plate.name
else:
    plate_name = new_plate_name
    
results_plate = PlateI()
results_plate.name = rstring(plate_name)
results_plate = conn.getUpdateService().saveAndReturnObject(results_plate)
results_plate_id = results_plate.getId()
results_dataset_id = ezomero.post_dataset(conn, "TempData", project_id, description="Temp dataset for image results")
results_dataset = conn.getObject("Dataset", results_dataset_id)

# Links new Plate with new Screen
link = ScreenPlateLinkI()
link.setParent(ScreenI(screen_id, False))
link.setChild(PlateI(results_plate_id, False))
link_update_service = conn.getUpdateService()
link_update_service.saveObject(link)

In [11]:
############# 2. Prepare image information #############
# Find image results to upload
results = [str(f) for f in pathlib.Path(saving_path).glob("*")]

image_results = [x for x in results if x.endswith((".png", ".npy", ".tiff"))]
image_result_tags = sorted(list(set([x.strip(".png.npy.tiff").split("_")[-1] for x in image_results])), key=lambda x: x.lower())
image_ids = sorted(list(set([x.split("_")[0] for x in image_results])), key=lambda x: x.lower())
print("Resulting image types:", image_result_tags)
print(f"You analysed {len(image_ids)} images.")

# Result measurements
table_results = [x for x in results if x.endswith(".csv")]


Resulting image types: ['CytoLabel', 'NucLabel', 'Overla', 'Ratio']
You analysed 1 images.


In [12]:
def load_result_image_from_disk(img_path):
    """
    Loads the image from disk using cv2, prepares the image axes and axis order to zctyx order that is expected by omero.
    Returns parent_id, and the image.
    """
    # TO DO: include loading multiple images with different tags

    _, tail = os.path.split(img_path)
    parent_id = os.path.splitext(tail)[0].split("_")[0]
    print("Parent image id:", parent_id)
    image = cv2.imread(img_path.__str__(), -1)  # TO DO: Adjust to different file formats
    #print(image.shape)
    # TO DO: include check how many axes are there
    
    add_axes = 5 - len(image.shape) 
    for axes in range(add_axes):
        image = np.expand_dims(image, axis=-1)  # adds axes to 2D images

    #print(image.shape)
    image = image.swapaxes(0, 3).swapaxes(1, 4).swapaxes(0, 1).swapaxes(1, 2)
    #print(image.shape)# Re-organise array from xyczt to zctyx order expected by OMERO

    return parent_id, image

In [13]:
OMEROWEB = "https://omero-imaging.uni-muenster.de/webclient/"

In [14]:
############# 3. Main Upload #############
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)

# Upload all result images
omero_images = []

for img_path in image_results:#pathlib.Path(saving_path).glob((f"(*.tiff)")):
    parent_id, image = load_result_image_from_disk(img_path)
    #print(parent_id)
    image_link = "Original Image: " + OMEROWEB + "?show=image-" + parent_id
    #omero_image = upload_image_from_npseq(image, img_path, conn, results_dataset, image_link)

    omero_image = cp_omero.upload_image_from_npseq(image, img_path, conn, results_dataset, image_link)
    omero_images.append(omero_image)

cp_omero.add_images_to_plate(omero_images, plate_id, results_plate_id, conn, results_dataset)

Parent image id: 186174
Image uploaded: 352892 : 186174_CytoLabel.tiff
Parent image id: 186174
Image uploaded: 352893 : 186174_NucLabel.tiff
Parent image id: 186174
Image uploaded: 352894 : 186174_Overlay.png
Parent image id: 186174
Image uploaded: 352895 : 186174_Ratio.png
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully up

You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val 

You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val = 1757
}.
You succesfully uploaded all images to the new plate object #0 (::omero::RLong)
{
    _val 

## 3. Tag Upload

To aid filtering inside omero, we will add tags to the result images based on their appendix. 
First, we query omero for all existing tags. 
Then, well find the uploaded images and add their corresponding tag to them (e.g. "NucleiSeg" for the nuclei-segmentation images)

In [15]:
#conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)

# Create dictionary to save your existing tags
existing_tags = {}

# Define your sql query, you use an sql to search for all existing tags, to prevent creation of double tags
sql = f"SELECT ann.id, ann.description, ann.textValue from TagAnnotation ann WHERE ann.details.owner.id = {tag_owner_id}"

for element in conn.getQueryService().projection(sql, None):    #element: list with 3 elements (ann.id, ann.description, ann.textValue)
                                                                #element[0]: object #0 (::omero::RLong){_val = 15286} type: <class 'omero.rtypes.RLongI'>
    tag_id, description, text = list(map(unwrap, element))
    existing_tags[text] = tag_id

print(f"The following tags exist: {existing_tags}.")

The following tags exist: {'Label_image': 15286, 'Processed': 15290, 'WGA_label': 15298, 'Nuclei_label': 15299, 'Segmentation': 15301, 'Test': 15303, 'Cyto_tag': 20543, 'Ratio_Image': 50712, 'CytoLabel': 50959, 'Ratio': 50962, 'NucLabel': 50958, 'Color': 50961, 'Overlay': 51026, '': 87282}.


In [16]:
#conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)
plate = conn.getObject("Plate", results_plate_id)

for well in plate.listChildren():
    index = well.countWellSample()
    for index in range(0, index):
        tag_name = well.getImage(index).getName().split(".")[0].split("_")[-1]
        if tag_name in existing_tags:
            try:
                tag_id = existing_tags[tag_name]
                image = conn.getObject("Image", well.getImage(index).getId())
                image.linkAnnotation(conn.getObject("Annotation", tag_id))
                print(f"Image {image.getName()} was tagged with {tag_name}")
            except omero.ValidationException:
                print(f"Image {image.getName()} was already tagged.")
        else:
            tag_ann = omero.gateway.TagAnnotationWrapper(conn)
            tag_ann.setValue(tag_name)
            tag_ann.setDescription("No description")
            tag_ann.save()
            image = conn.getObject("Image", well.getImage(index).getId())
            image.linkAnnotation(tag_ann)
            existing_tags[tag_name] = tag_ann.id
            print("New tag created: ", tag_name, ".")

Image 186174_CytoLabel.tiff was tagged with CytoLabel
Image 186174_NucLabel.tiff was tagged with NucLabel
Image 186174_Overlay.png was tagged with Overlay
Image 186174_Ratio.png was tagged with Ratio


In [17]:
# Add key:value pairs to your data:
# You can create a simple annotation dictionary to add Key:Value pairs to the plate

#annotation_dict = {"TiM23": "WS44", "Software": "Cellprofiler 4.2.5", "Segmentation Algorithm": "Cellpose"} 

# Add KV pairs to the plate:
map_ann_id = ezomero.post_map_annotation(conn, "Plate", results_plate_id.getValue(), annotation_dict, "myns")

# Add KV pairs to every image in the plate:
results_plate = conn.getObject("Plate", results_plate_id)

for well in results_plate.listChildren():
    for image in well.listChildren():
        map_ann_id = ezomero.post_map_annotation(conn, "Image", image.id, annotation_dict, "myns")
        
print(f"You added the these annotations {annotation_dict} as key:value pairs to the wells.")

object group 103
object group 103
object group 103
object group 103
object group 103
You added the these annotations {'Goal & Description': 'We segment the cells and calculate the YAP/Taz ratio in the nuclei.', 'Software': 'Cellprofiler', 'Software Version': '4.2.5', 'Segmentation Algorithm': 'Cellpose', 'Segmentation Algorithm Version': '?', 'Models': 'nuclei, cyto2', 'eLabTFW': 'https://eln.uni-muenster.de/experiments.php?mode=edit&id=71'} as key:value pairs to the wells.


In [18]:
# Upload pipeline to results plate:
filepath_pipeline_txt = f"{saving_path}\Omero_Pipeline.cppipe"
file_ann_id = ezomero.post_file_annotation(conn, "Plate", results_plate_id.getValue(), filepath_pipeline_txt, ns= "myns", description="This pipeline was used for analysis.")

print("You succesfully added your pipeline as file annotation to the plate.")

You succesfully added your pipeline as file annotation to the plate.


## 4. Upload of ROIs

In [31]:
def load_labelimages(img_path):
    _, tail = os.path.split(img_path)
    parent_id = os.path.splitext(tail)[0].split("_")[0]
    print("Parent image id:", parent_id)
    image = cv2.imread(img_path.__str__(), -1)
    
    return parent_id, image

# Create contours that will be uploaded as ROIs from label images
def create_contours(labelimage):
    contour_dict = {}
    for i in range(1, labelimage.max()+1):
        mask_ = ma.equal(labelimage, i)
        if np.any(mask_):
            contour = find_contours(mask_)
            contour_dict[str(i)] = contour[0]
            
    return contour_dict


def upload_ROIs(contour_dict, parent_id, conn):
    roi_dict = {}     # {"grey_value":roi_id}
    for key, value in contour_dict.items():
    # create polygon shape for each
        shape = [ezomero.rois.Polygon(np.flip(value), label=key,
                fill_color=(0,0,0,0), stroke_color=(255, 255, 0, 255))]  
            #expects a list of tuples of floats, label has to be grey-value
            #as label of shape is displayed as ROI-name in OMERO.iviewer
    
        # create roi and link shape to roi
        roi_id = ezomero.post_roi(conn, int(parent_id), shape, name = key)
        # create dict grey_value : Roi_ID
        roi_dict[key] = roi_id
        
    return roi_dict

In [20]:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)

In [21]:
# Upload all result images
parent_ids = []
labelimages = []

#find solution to split nuclei and cytoplasm

for img_path in pathlib.Path(saving_path).glob(("*NucLabel.tiff")):
    parent_id, image = load_labelimages(img_path)
    parent_ids.append(parent_id)
    labelimages.append(image)

Parent image id: 186174


In [32]:
roiid_grayvalue_dict = []
for index, image in enumerate(labelimages):
    contour_dict = create_contours(image)
    roi_dict = upload_ROIs(contour_dict, parent_ids[index], conn)
    roiid_grayvalue_dict.append(roi_dict)

## 5. Upload result as omero.table

Finally, we will upload the measurement results as an "omero.table" to Omero and link it to the analysed plate. 
These measurement results can be viewed in omero.parade-crossfilter.

In [23]:
for img_path in pathlib.Path(saving_path).glob(("*Nuclei_Seg_Relate.csv")):
    print(img_path)

c:\users\min_acc1\appdata\local\temp\5\tmpn16qqres\186174_Nuclei_Seg_Relate.csv


In [41]:
results_csv = pd.read_csv(img_path, encoding="Latin1")
#results_csv.columns

In [40]:
#df

In [34]:
df = pd.DataFrame.from_dict(roiid_grayvalue_dict).T.reset_index()
df["index"] = pd.to_numeric(df["index"])
df.rename(columns={'index': 'ObjectNumber', 0: 'ROIID'}, inplace=True)
df["Image_ID"] = int(parent_id)
df["Well_ID"] = 26775

In [43]:
results_csv

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,...,Location_CenterMassIntensity_Z_YapTaz,Location_Center_X,Location_Center_Y,Location_Center_Z,Location_MaxIntensity_X_YapTaz,Location_MaxIntensity_Y_YapTaz,Location_MaxIntensity_Z_YapTaz,Math_YapTazRatio,Number_Object_Number,Parent_Nuclei_Seg
0,1,1,333,500,295,39,270,19,282.009009,28.606607,...,0.0,282.009009,28.606607,0,277.0,25.0,0.0,0.576986,1,1
1,1,2,414,616,96,129,68,107,81.594203,117.422705,...,0.0,81.594203,117.422705,0,87.0,115.0,0.0,0.505644,2,5
2,1,3,466,616,469,172,447,144,457.476395,157.538627,...,0.0,457.476395,157.538627,0,464.0,157.0,0.0,0.646913,3,6
3,1,4,426,588,151,189,130,161,140.436620,174.561033,...,0.0,140.436620,174.561033,0,139.0,176.0,0.0,0.368525,4,7
4,1,5,224,300,215,233,195,218,204.660714,225.040179,...,0.0,204.660714,225.040179,0,213.0,226.0,0.0,0.226648,5,8
5,1,6,353,504,217,286,196,262,205.957507,273.878187,...,0.0,205.957507,273.878187,0,210.0,276.0,0.0,0.433613,6,9
6,1,7,425,550,288,295,263,273,275.388235,283.574118,...,0.0,275.388235,283.574118,0,280.0,281.0,0.0,0.722860,7,10
7,1,8,259,484,528,301,506,279,517.065637,289.494208,...,0.0,517.065637,289.494208,0,516.0,284.0,0.0,0.424060,8,11
8,1,9,278,391,385,301,362,284,373.687050,292.014388,...,0.0,373.687050,292.014388,0,377.0,290.0,0.0,0.374948,9,12
9,1,10,369,480,466,319,446,295,455.498645,306.504065,...,0.0,455.498645,306.504065,0,458.0,305.0,0.0,0.768965,10,13


In [35]:
mytable = pd.merge(right=results_csv, left=df, right_on="ObjectNumber", left_on="ObjectNumber")

In [36]:
# Here we create the columns with the correct column types for an omero.table
cols = []

for col in mytable.columns:
    if col == "Image_ID":
        cols.append(ImageColumn(col, '', mytable[col])) #Evtl column mit "imagename einfügen?"
    elif col == "ROIID":
        cols.append(RoiColumn(col, '', mytable[col]))
    elif col == "Well_ID":
        cols.append(WellColumn(col, '', mytable[col]))
    elif mytable[col].dtype == 'int64':
        cols.append(LongColumn(col, '', mytable[col]))
    elif mytable[col].dtype == 'float64':
        cols.append(DoubleColumn(col, '', mytable[col]))

In [37]:
# Connection Check:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)
print(conn.isConnected())

True


In [38]:
mytable

,ObjectNumber,ROIID,Image_ID,Well_ID,ImageNumber,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,...,Location_CenterMassIntensity_Z_YapTaz,Location_Center_X,Location_Center_Y,Location_Center_Z,Location_MaxIntensity_X_YapTaz,Location_MaxIntensity_Y_YapTaz,Location_MaxIntensity_Z_YapTaz,Math_YapTazRatio,Number_Object_Number,Parent_Nuclei_Seg
0,1,142264,186174,26775,1,333,500,295,39,270,...,0.0,282.009009,28.606607,0,277.0,25.0,0.0,0.576986,1,1
1,2,142265,186174,26775,1,414,616,96,129,68,...,0.0,81.594203,117.422705,0,87.0,115.0,0.0,0.505644,2,5
2,3,142266,186174,26775,1,466,616,469,172,447,...,0.0,457.476395,157.538627,0,464.0,157.0,0.0,0.646913,3,6
3,4,142267,186174,26775,1,426,588,151,189,130,...,0.0,140.436620,174.561033,0,139.0,176.0,0.0,0.368525,4,7
4,5,142268,186174,26775,1,224,300,215,233,195,...,0.0,204.660714,225.040179,0,213.0,226.0,0.0,0.226648,5,8
5,6,142269,186174,26775,1,353,504,217,286,196,...,0.0,205.957507,273.878187,0,210.0,276.0,0.0,0.433613,6,9
6,7,142270,186174,26775,1,425,550,288,295,263,...,0.0,275.388235,283.574118,0,280.0,281.0,0.0,0.722860,7,10
7,8,142271,186174,26775,1,259,484,528,301,506,...,0.0,517.065637,289.494208,0,516.0,284.0,0.0,0.424060,8,11
8,9,142272,186174,26775,1,278,391,385,301,362,...,0.0,373.687050,292.014388,0,377.0,290.0,0.0,0.374948,9,12
9,10,142273,186174,26775,1,369,480,466,319,446,...,0.0,455.498645,306.504065,0,458.0,305.0,0.0,0.768965,10,13


In [39]:
# Initialize a table
resources = conn.c.sf.sharedResources()
repository_id = resources.repositories().descriptions[0].getId().getValue()
table_name = plate_name +"_CellprofilerResults_ROIs"
table = resources.newTable(repository_id, table_name)
table.initialize(cols)
table.addData(cols)

# Create file annotation
orig_file = table.getOriginalFile()
file_ann = FileAnnotationWrapper(conn)
file_ann.setNs(NSBULKANNOTATIONS)
file_ann._obj.file = OriginalFileI(orig_file.id.val, False)
file_ann.save()

# Link the table to the original screen
screen.linkAnnotation(file_ann)
table.close()
print("You added your analysis results as omero.table to your screen. You can now view them in omero.parade crossfilter.")

You added your analysis results as omero.table to your screen. You can now view them in omero.parade crossfilter.


### 5. Clean Up

In [ ]:
# Delete temporay results data set
conn.deleteObjects("Dataset", [results_dataset_id])

In [ ]:
# Close the omero connection
conn.close()

In [ ]:
# Delete your temporary directory
if output_dir == "temp_dir":
    shutil.rmtree(temp_dir)